In [1]:
import requests, json
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import asyncio
import os
import json
import copy

from anchorpy import Wallet
from anchorpy import Provider
from solana.keypair import Keypair
from solana.rpc.async_api import AsyncClient

from driftpy.constants.config import configs
from driftpy.types import *
#MarketType, OrderType, OrderParams, PositionDirection, OrderTriggerCondition

from driftpy.clearing_house import ClearingHouse
from driftpy.constants.numeric_constants import BASE_PRECISION, PRICE_PRECISION

from driftpy.constants.config import configs
from anchorpy import Provider
from anchorpy import Wallet
from solana.rpc.async_api import AsyncClient
from driftpy.clearing_house import ClearingHouse
from driftpy.clearing_house_user import ClearingHouseUser
from driftpy.accounts import *
from solana.keypair import Keypair
from driftpy.math.positions import is_available
from driftpy.constants.numeric_constants import *
import os
from driftpy.clearing_house_user import ClearingHouseUser
from driftpy.clearing_house import ClearingHouse
from solana.keypair import Keypair
from driftpy.constants.config import configs
import os
from anchorpy import Wallet, Provider
from solana.rpc.async_api import AsyncClient




MARKET_INDEX_SOL = 0
userAccount = "ECbtn9Y34m6L89EBDMRYxhpyrFgSHEcbELgozpqAvcDY"
PKEY = "7f6d3DWGkNrKTERtW9McbBQCVbZibuHUf7CsdUjJyK1t"
URL_ORDERS_SELF   = f"https://master.api.drift.trade/orders/userAccounts?userPublicKeys={PKEY}&pageIndex=0&pageSize=100&marketType=perp"
## (below), putting public key only allows me to see
URL_HISTORY_SELF  = f"https://beta.drift.trade/overview/history/tradeHistory?userAccount={PKEY}"
URL_DEVNET_OB     = "https://master.dlob.drift.trade/orders/json"

In [2]:
from driftpy import constants
constants.markets.devnet_markets

[Market(symbol='SOL-PERP', base_asset_symbol='SOL', market_index=0, pyth_oracle=J83w4HKfqxwcq3BEMMkPFSppX3gqekLyLJBexebFVkix),
 Market(symbol='BTC-PERP', base_asset_symbol='BTC', market_index=1, pyth_oracle=HovQMDrbAgAYPCmHVSrezcSmkMtXSSUsLDFANExrZh2J),
 Market(symbol='ETH-PERP', base_asset_symbol='ETH', market_index=2, pyth_oracle=EdVCmQ9FSPcVe5YySXDPCRmc8aDQLKJ9xvYBMZPie1Vw)]

In [3]:
def orderbook_all(perp_market="SOL-PERP"):
    """ Queries DEVNET orderbok for specified perp market, organizes data into bid/asks sorted
    """
    try:
        r = requests.get(URL_DEVNET_OB).json()
    except:
        print(r)
        raise Exception("Unable to fetch orderbook")

    slot    = r['slot']
    oracles = r['oracles']
    orders  = r['orders']

    ## market index -> price
    market_to_oracle_map = pd.DataFrame(r['oracles']).set_index('marketIndex').to_dict()['price']

    ## Get list of outstanding orders    
    orders_all = list(map(lambda d: {"user":d['user'][:10], **d.pop('order')}, orders))
    df = pd.DataFrame(orders_all)
    df_raw = pd.DataFrame(orders_all)
    
    ## Only interested in (a) SOL-PERP (b) limit orders
    # SOL is marketINdex=0
    df = df[(df.marketIndex == MARKET_INDEX_SOL) & (df.marketType=='perp')]
    
    df["oraclePrice"] = df["marketIndex"].apply(lambda x: market_to_oracle_map.get(x, None))

    ## Convert from lamports:
    for col in ['price', 'oraclePrice', 'oraclePriceOffset']:
        df[col] = df[col].astype(int)
        df[col] /= 1e6

    for col in ['quoteAssetAmountFilled']:
        df[col] = df[col].astype(int)
        df[col] /= 1e6 

    for col in ['baseAssetAmount', 'baseAssetAmountFilled']:
        df[col] = df[col].astype(int)
        df[col] /= 1e9
        
    
    bid_df = df[(df.direction == 'long') & (df.orderType == 'limit')]
    ask_df = df[(df.direction == 'short')   &( df.orderType == 'limit')]
    
    ## Fixed prices are ok
    ## Floating prices need to be marked to a fixed price according to oracle price    
    bid_float = bid_df.loc[bid_df.price == 0] # inv: 0 price <-> floating order .w.r.t oracle 
    ask_float = ask_df.loc[ask_df.price == 0] #
    
    bid_df.loc[bid_df.price == 0, "price"] = bid_float["oraclePrice"] + bid_float["oraclePriceOffset"]
    ask_df.loc[ask_df.price == 0, "price"] = ask_float["oraclePrice"] + ask_float["oraclePriceOffset"]
    
    
    bid_df = bid_df.sort_values(['price'], ascending=False)
    ask_df = ask_df.sort_values(['price'])
    
    bid_df = bid_df.reset_index(drop=True)
    ask_df = ask_df.reset_index(drop=True)
    
    return df_raw, bid_df, ask_df

def orderbook_expanded(perp_market="SOL-PERP"):
    """
        simplieid view of orderbook
    """
    _, bid, ask = orderbook_all(perp_market)
    
    
    
    bid_simple = bid[["price", "baseAssetAmount", "baseAssetAmountFilled", "postOnly", "oraclePriceOffset", "oraclePrice"]]
    ask_simple = ask[["price", "baseAssetAmount", "baseAssetAmountFilled", "postOnly", "oraclePriceOffset", "oraclePrice"]]

    #### SOME ARE PARTIALLY FILLED
    bid_simple["qty"] = bid_simple["baseAssetAmount"] - bid_simple["baseAssetAmountFilled"]
    ask_simple["qty"] = ask_simple["baseAssetAmount"] - ask_simple["baseAssetAmountFilled"]
    

    return bid_simple, ask_simple

def get_dlob(perp_market="SOL-PERP"):
    """
        Returns simplified orderbook view 
    """
    bid, ask = orderbook_expanded(perp_market)
    
    bid_ob = bid_s[["price", "qty"]].groupby("price", sort=False).sum()
    ask_ob = ask_s[["price", "qty"]].groupby("price", sort=False).sum()
    return bid_ob, ask_ob
    
def fair_price_dlob(bid_ob, ask_ob):
    return (bid_ob.index.max() + ask_ob.index.min() ) / 2.0
    

df_raw, bid, ask = orderbook_all()
print(f"#bids={len(bid)}, #asks={len(ask)}")
bid_s, ask_s     = orderbook_expanded()
bid_ob, ask_ob   = get_dlob()

#bids=16, #asks=4


In [5]:
async def get_solana_perp_oracle():
    """ return SOL-PERP oracle """
    with open(os.path.expanduser("/home/sjl/.config/solana/key.json"), 'r') as f: secret = json.load(f)
    kp = Keypair.from_secret_key(bytes(secret))
    subaccount = 0
    print('using public key:', kp.public_key, " subaccount = ", subaccount)

    env = 'devnet'
    config = configs[env]
    wallet = Wallet(kp) 
    connection = AsyncClient(config.default_http)
    provider = Provider(connection, wallet)

    ch = ClearingHouse.from_config(config, provider)
    chu = ClearingHouseUser(ch, authority=ch, subaccount_id=subaccount)
    perp_market = await chu.get_perp_market(0)
    oracle = (await chu.get_perp_oracle_data(perp_market)).price / PRICE_PRECISION

    return oracle

In [6]:

def mm_order_single(order_offset, order_direction=PositionDirection.LONG(), order_qty=0):

    # order_base_qty  = 100
    # order_direction = PositionDirection.LONG()
    # order_oracle_offset = 1e-4 * PRICE_PRECISION 

    return OrderParams(
        order_type=OrderType.LIMIT(),
        market_type=MarketType.PERP(),
        direction=order_direction,
        user_order_id=0,
        base_asset_amount = int(order_qty * BASE_PRECISION),
        price=0,
        market_index=0,
        reduce_only=False,
        post_only=PostOnlyParams.TRY_POST_ONLY(),
        immediate_or_cancel=False,
        trigger_price=0,
        trigger_condition=OrderTriggerCondition.ABOVE(),
        oracle_price_offset= int(order_offset * PRICE_PRECISION),
        auction_duration=None,
        max_ts=None,
        auction_start_price=None,
        auction_end_price=None,
    )


async def construct_and_place(bid_offsets, bid_qtys, ask_offsets, ask_qtys, cancel_existing=True):
    """
        bundles offsets / quantities for bids / asks into an array of orders
        and places order
        
        cancels existing orders before placing
    """
    all_orders = []
    
    if(cancel_existing):
        all_orders.append(await drift_acct.get_cancel_orders_ix(0))
    
    for bo, bq in zip(bid_offsets, bid_qtys):
        temp_order = mm_order_single(bo, PositionDirection.LONG(), bq)
        all_orders.append(
            await drift_acct.get_place_perp_order_ix(temp_order, 0)
        )
    for ao, aq in zip(ask_offsets, ask_qtys):
         all_orders.append(
             await drift_acct.get_place_perp_order_ix(mm_order_single(ao, PositionDirection.SHORT(), aq), 0)
         )
        
    ret = await drift_acct.send_ixs(all_orders)
    
    return ret


bid_offsets = [-10, -12, -15]
bid_qtys    = [1.1, 2.2, 1.1]

ask_offsets = [10, 12, 15]
ask_qtys    = [1.1, 2.2, 1.1]

# await construct_and_place(bid_offsets, bid_qtys, ask_offsets, ask_qtys)

# Market Making Parameters

In [7]:
## Set up clearing house user client to view balances 

subaccount_id = 0
env = 'devnet'
url = 'https://api.devnet.solana.com'
with open(os.path.expanduser("/home/sjl/.config/solana/key.json"), 'r') as f: secret = json.load(f)
kp = Keypair.from_secret_key(bytes(secret))
config = configs[env]
wallet = Wallet(kp)
connection = AsyncClient(url)
provider = Provider(connection, wallet)
drift_acct = ClearingHouse.from_config(config, provider)
chu        = ClearingHouseUser(drift_acct)

/home/sjl/Projects/drift/driftpy-examples/venv/lib/python3.10/site-packages/driftpy/idl/drift.json


In [9]:
await chu.get_unrealized_pnl() / PRICE_PRECISION

3.525205679999828

In [13]:
await chu.get_user_position(0)

PerpPosition(last_cumulative_funding_rate=-3291315921, base_asset_amount=-109960000000, quote_asset_amount=2531636678, quote_break_even_amount=2531474468, quote_entry_amount=2530827537, open_bids=4400000000, open_asks=-4400000000, settled_pnl=0, lp_shares=0, last_base_asset_amount_per_lp=0, last_quote_asset_amount_per_lp=0, remainder_base_asset_amount=0, market_index=0, open_orders=6, padding=ListContainer([0]))

In [ ]:
# Constants that never change 

## Initial deposit into account
INITIAL_COLLATERAL_SOL = 1
INITIAL_COLLATERAL_USD = 100

TARGET_MAX_SIZE = 100
LEVERAGE_LIMIT  = 2

# Computed from above:
curr_collateral = None
curr_leverage   = None

AGGRESSION = 10e-4



In [ ]:
bid_ob.reset_index().iloc[0]

In [ ]:
# fair_price, AGGRESSION
ask_width      = AGGRESSION  * 0.8  # TODO

ask_prices     = np.linspace(fair_price, fair_price*(1+ask_width), 3)
ask_prices

In [ ]:
## Below specific for SOL-PERP

epoch = 0

while epoch < 3:
    
    print("Epoch: ", epoch)
    epoch += 1
    
    ## PART 1: Record state change from previous state
    #  - See if any fills
    #  - Get current oracle / market price, mark collateral and leverage properly
    
    # Current SOL-PERP position
    pp = await chu.get_user_position(0)
    current_pos = pp.base_asset_amount / 1e9  
    print(f"> current pos = {current_pos}")
    
    # (1): Get the current order book (GET Request)
    bid_ob, ask_ob = get_dlob()

    # (2): Calculate the fair price based on the order book
    fair_price = fair_price_dlob(bid_ob, ask_ob)
    print(f"> fair price = {fair_price}")
    
    ## PART 2: Prepare new bids / ask price points
    
    # ------ bid side -------
    ## Compute total size
    bid_target_pos     = TARGET_MAX_SIZE - current_pos
    bid_offset_width   = AGGRESSION * 1.1  # TODO
    
    bid_offsets        = np.linspace(0, bid_offset_width, 4)[1:]
    bid_qts            = bid_target_pos * np.array([0.25, 0.5, 0.25])
    
    print(f"> bid_offsets = {bid_offsets}")
    print(f"> bid_qts = {bid_qts}")
    
    # ------ ask side -------
    ask_target_pos = TARGET_MAX_SIZE - current_pos
    ask_offset_width      = AGGRESSION  * 0.8  # TODO
    
    ask_offsets     = np.linspace(0, ask_offset_width, 4)[1:]
    ask_qts        = ask_target_pos * np.array([0.25, 0.5, 0.25])
    
    print(f"> ask_prices = {ask_offsets}")
    print(f"> ask_qts = {ask_qts}")
    
    
    ## PART 3: Send request via API for new order
    await construct_and_place(bid_offsets, bid_qts, ask_offsets, ask_qts)
    
    ## Log what has happened in a CSV

    
    
    
    
    # Step 6: Wait for 1 second before repeating the loop
    await asyncio.sleep(10)
